You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os

os.environ["OPENAI_MODEL_NAME"] = 'Llama-4-Maverick-17B-128E-Instruct-FP8'
os.environ["OPENAI_API_BASE"] = 'https://api.llama.com/compat/v1/'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

2025-08-18 13:11:51,541 - 123145518489600 - __init__.py-__init__:168 - ERROR: Exception while exporting Span.
Traceback (most recent call last):
  File "/Users/esalum/Documents/Dev/Multi-AI-Agent-Systems-with-crewAI/lib/python3.12/site-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/esalum/Documents/Dev/Multi-AI-Agent-Systems-with-crewAI/lib/python3.12/site-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/Users/esalum/Documents/Dev/Multi-AI-Agent-Systems-with-crewAI/lib/python3.12/site-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
TimeoutError: timed out

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/esalum/Documents/Dev/Multi-AI-Agent-Systems-with-crewAI/lib/python3.12/site-packages/urllib3/connectionpool.py", line 787, in ur

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
To create a comprehensive content plan document on Artificial Intelligence (AI), I will start by researching the latest trends, key players, and noteworthy news in the AI domain. This will help in understanding the current landscape and identifying relevant information for the target audience.

1. **Latest Trends and News in AI:**
   - The rise of Explainable AI (XAI) for transparency in AI decision-making.
   - Advancements in Natural Language Processing (NLP) and its applications.
   - Increased adoption of AI in healthcare for diagnostics and personal

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# Navigating the Future of Artificial Intelligence: Trends, Applications, and Challenges

Artificial Intelligence (AI) has been transforming industries and revolutionizing the way businesses operate. As we continue to witness significant advancements in this field, understanding the current trends, applications, and challenges of AI is crucial for leveraging its potential. In this blog post, we'll explore the latest developments in AI, its diverse applications, and the ethical considerations that come with it.

## The Rise of AI: Current Trends and Advancements

The field of AI is rapidly evolving, with several trends shaping its future. One significant advancement is the rise of Explainable AI (XAI), which aims to enhance transparency in AI decision-making processes. XAI is crucial for building trust in AI systems, especially in high-stakes applications like healthcare and finance. According to a report by McKinsey, the adoption of XAI is expected to increase as organizations seek to improve the accountability and reliability of their AI systems.

Another area witnessing significant advancements is Natural Language Processing (NLP). NLP has numerous applications, including language translation, sentiment analysis, and chatbots. Recent breakthroughs in NLP have enabled machines to understand and generate human-like language, opening up new possibilities for human-computer interaction. For instance, virtual assistants like Siri and Alexa rely heavily on NLP to understand voice commands and respond accordingly. These advancements are not only enhancing user experience but also streamlining various business processes.

The healthcare sector is also experiencing a significant transformation due to AI. AI-powered diagnostic tools are being used to detect diseases at an early stage, and personalized medicine is becoming increasingly prevalent. AI algorithms can analyze vast amounts of medical data to identify patterns and predict patient outcomes, enabling healthcare professionals to make more informed decisions. This integration of AI in healthcare is expected to improve patient care and outcomes significantly.

## Key Players and Innovations in AI

Major technology companies like Google, Microsoft, and Amazon are driving AI innovation, investing heavily in research and development. These companies are pushing the boundaries of what is possible with AI, from developing more sophisticated NLP models to creating AI-powered hardware. For example, Google's TensorFlow has become a popular open-source framework for building and deploying AI models. This has democratized access to AI technology, allowing a wider range of developers to build AI-powered applications.

Startups are also playing a crucial role in the AI ecosystem, focusing on niche applications and innovative solutions. These companies are often more agile than their larger counterparts, allowing them to quickly adapt to changing market conditions and capitalize on emerging trends. As a result, startups are driving innovation in areas like AI-powered cybersecurity and healthcare diagnostics, further expanding the scope of AI applications.

## Exploring AI Applications and Future Directions

AI has the potential to transform numerous industries, from finance and education to transportation and energy. In finance, AI-powered systems can detect anomalies in transaction data, helping to prevent financial crimes. In education, AI can personalize learning experiences, improving student outcomes and enhancing the overall quality of education. The versatility of AI makes it a valuable tool across various sectors.

As AI continues to evolve, we can expect to see significant societal impacts. For instance, AI-powered autonomous vehicles may revolutionize the transportation sector, reducing accidents and improving traffic flow. However, the increasing reliance on AI also raises concerns about job displacement and the need for workers to develop new skills. It is essential to address these challenges proactively to ensure a smooth transition.

## Addressing the Challenges and Ethical Considerations of AI

One of the significant challenges associated with AI is job displacement. As AI-powered automation becomes more prevalent, there is a risk that certain jobs may become obsolete. However, it's also possible that AI will create new job opportunities, especially in areas like AI development and deployment. To mitigate the negative impacts, it's crucial to invest in education and retraining programs that prepare workers for an AI-driven economy.

Another critical issue is bias in AI systems. AI algorithms can perpetuate existing biases if they are trained on biased data, leading to unfair outcomes. To address this issue, it's essential to develop ethical frameworks that prioritize fairness, transparency, and accountability. Techniques like debiasing word embeddings and using fairness metrics can help mitigate bias in AI systems. Ensuring that AI systems are fair and transparent is vital for building trust in these technologies.

## Getting Started with AI: Resources and Next Steps

For professionals and enthusiasts looking to learn more about AI, there are numerous resources available. Online courses like Andrew Ng's Machine Learning course and Stanford University's CS229: Machine Learning provide a comprehensive introduction to AI and machine learning. Books like "Deep Learning" by Ian Goodfellow, Yoshua Bengio, and Aaron Courville offer in-depth insights into the latest AI techniques. Staying updated with the latest research and developments is key to leveraging AI effectively.

Recent research papers and articles from tech journals like TechCrunch and The Verge can also provide valuable insights into the latest AI trends and advancements. By staying informed and continually updating their skills, professionals can remain at the forefront of AI innovation. This continuous learning approach is essential in the rapidly evolving field of AI.

## Conclusion and Call to Action

In conclusion, AI is a rapidly evolving field with significant potential to transform industries and revolutionize the way we live and work. By understanding the current trends, applications, and challenges of AI, we can unlock its full potential and create a more equitable and prosperous future. Whether you're a professional looking to integrate AI into your business or an enthusiast curious about AI's potential impacts, we encourage you to explore AI further and consider implementing AI solutions in your respective domains. With the right knowledge and resources, you can harness the power of AI to drive innovation and achieve your goals.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [13]:
topic = "AI Accelerators and their impact on startups"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on AI Accelerators and their impact on startups.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
To create a comprehensive content plan document on "AI Accelerators and their impact on startups," I will start by gathering and prioritizing the latest trends, key players, and noteworthy news on the topic. Then, I will identify the target audience and their interests. After that, I will develop a detailed content outline and include relevant SEO keywords and data.

First, let's prioritize the latest trends and key players in AI Accelerators:

1. **Latest Trends:**
   - Increased adoption of AI accelerators in edge devices.
   - Gro

In [14]:
Markdown(result)

# The Revolutionary Impact of AI Accelerators on Startups
## Introduction
The world of technology is undergoing a significant transformation with the advent of Artificial Intelligence (AI). At the forefront of this revolution are AI accelerators, specialized hardware designed to enhance the performance of AI and Machine Learning (ML) tasks. As AI continues to permeate various industries, understanding the role of AI accelerators becomes crucial, especially for startups looking to leverage AI for innovation and competitive advantage. This blog post explores how AI accelerators are revolutionizing the startup landscape by enhancing performance, reducing costs, and enabling new AI-driven products and services.

The advent of AI accelerators represents a paradigm shift in how AI computations are performed, moving away from general-purpose CPUs to more specialized hardware. With the growing importance of AI in business and technology, the demand for efficient, powerful, and cost-effective AI processing is on the rise. Startups, in particular, are poised to benefit significantly from AI accelerators as they look to differentiate themselves and deliver cutting-edge solutions.

In this blog post, we will delve into the world of AI accelerators, examining their types, benefits, and the impact they have on startups. We will also explore the latest trends, key players, and the challenges startups might face in adopting AI accelerators.

## What are AI Accelerators?
AI accelerators are specialized hardware components designed to accelerate AI and ML computations. Unlike general-purpose CPUs, AI accelerators are optimized for the complex mathematical operations involved in deep learning and other AI tasks. The primary types of AI accelerators include Graphics Processing Units (GPUs), Tensor Processing Units (TPUs), Field-Programmable Gate Arrays (FPGAs), and Application-Specific Integrated Circuits (ASICs).

Each type of AI accelerator has its unique strengths and is suited for different applications. For example, GPUs are widely used for their versatility and high performance in both graphics rendering and AI computations. TPUs, on the other hand, are custom-built by Google for high-speed ML computations, particularly for its TensorFlow framework. Understanding these different types of AI accelerators is crucial for startups to choose the right hardware for their specific AI needs.

By offloading complex AI tasks from general-purpose CPUs, AI accelerators significantly improve performance and efficiency. This not only speeds up the development and deployment of AI models but also enables startups to handle larger, more complex datasets.

## Impact of AI Accelerators on Startups
AI accelerators are having a profound impact on startups by enhancing performance and efficiency in AI tasks. For startups, being able to quickly develop, train, and deploy AI models is a significant competitive advantage. AI accelerators enable this by providing the necessary computational power to handle complex AI computations at scale.

Moreover, AI accelerators are reducing costs and power consumption for AI operations. By optimizing AI computations, startups can achieve the same or better results using fewer resources. This is particularly important for early-stage startups where resource efficiency can be a deciding factor in survival and success.

Furthermore, AI accelerators are enabling the development of new AI-driven products and services. With the enhanced computational capabilities provided by AI accelerators, startups can explore new AI applications that were previously not feasible due to computational constraints. This opens up new markets and opportunities for startups to innovate and grow.

## Latest Trends and Innovations
The landscape of AI accelerators is rapidly evolving, with several emerging trends and innovations. One significant trend is Edge AI, which involves processing AI tasks at the edge of the network, closer to the source of the data. Edge AI reduces latency and improves real-time decision-making, making it crucial for applications like autonomous vehicles and IoT devices.

Another trend is the development of custom AI accelerators for specific tasks. Companies are designing specialized AI accelerators tailored to their unique AI workloads, offering significant performance and efficiency gains. The integration of AI accelerators with 5G is another emerging trend, enabling new use cases that require rapid data processing and analysis.

## Key Players and Market Dynamics
The AI accelerator market is characterized by a mix of established players and innovative startups. Companies like NVIDIA, Google, and Intel are leading the charge with their advanced AI accelerator technologies. NVIDIA's GPUs, for example, are widely used in AI research and applications, while Google's TPUs are renowned for their performance in ML computations.

Emerging startups are also making significant contributions to the AI accelerator landscape. These companies are often focused on niche areas or innovative approaches to AI acceleration, providing startups with a range of options to choose from. The market dynamics are competitive, with companies continually innovating to improve performance, efficiency, and cost-effectiveness.

## Challenges and Considerations for Startups
While AI accelerators offer numerous benefits, startups must carefully consider several challenges when adopting these technologies. Choosing the right AI accelerator for specific needs is crucial. Startups need to evaluate their AI workloads and select an AI accelerator that best matches their requirements.

Integration and compatibility issues with existing infrastructure are another consideration. Startups must ensure that their chosen AI accelerator can seamlessly integrate with their current hardware and software environments. Cost considerations and ROI analysis are also vital to ensure that the benefits of AI accelerators outweigh the costs.

## Future Outlook and Opportunities
The future of AI accelerators is bright, with emerging trends and future directions promising to further revolutionize the AI landscape. As AI continues to evolve, the demand for more powerful, efficient, and specialized AI accelerators will grow. Startups that invest in AI accelerators today are positioning themselves for success in a future where AI is increasingly ubiquitous.

For startups, there are significant opportunities in both leveraging existing AI accelerators and developing innovative AI acceleration solutions. Whether it's improving AI performance, reducing costs, or enabling new AI applications, AI accelerators are set to play a pivotal role in the success of startups in the AI era.

## Conclusion and Call to Action
In conclusion, AI accelerators are revolutionizing the startup landscape by enhancing performance, reducing costs, and enabling new AI-driven products and services. As the AI landscape continues to evolve, understanding and leveraging AI accelerators will be crucial for startups looking to innovate and succeed.

We encourage startups to explore the potential of AI accelerators and invest in these technologies to drive their AI initiatives forward. With the right AI accelerator, startups can unlock new opportunities, achieve competitive advantage, and thrive in the AI-driven future.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).